In [4]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [7]:
data = pd.read_csv('2003_mo_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego,Rio De Janeiro - Consumo de Cimento (t)
0,2003-1,0.291137,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,199.081280,0.773489,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,8.356512,254.362
1,2003-2,0.293538,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,198.561780,0.773547,3.587441e+08,1.814108e+07,22.149228,3.474056e+08,8.350332,252.842
2,2003-3,0.295914,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,198.773336,0.773606,3.591452e+08,1.814681e+07,22.154735,3.475266e+08,8.344152,229.385
3,2003-4,0.298264,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,198.753800,0.773665,3.595464e+08,1.815255e+07,22.160243,3.476476e+08,8.337971,243.407
4,2003-5,0.300588,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,198.721455,0.773724,3.599475e+08,1.815828e+07,22.165750,3.477686e+08,8.331791,256.535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.559828,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.917513,NaN,NaN,NaN,NaN,NaN,NaN,241.702
236,2022-9,0.558289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.658000,NaN,NaN,NaN,NaN,NaN,NaN,221.036
237,2022-10,0.556367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232.339116,NaN,NaN,NaN,NaN,NaN,NaN,224.399
238,2022-11,0.554021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,231.667333,NaN,NaN,NaN,NaN,NaN,NaN,221.319


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.438978,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-0.476880,0.839598,1.110414,-1.635096,-1.325228,-1.036232,1.324172
188,1.443990,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-0.460898,0.793634,1.098172,-1.625552,-1.326867,-1.053116,1.322796
189,1.457014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-0.421271,0.747671,1.085930,-1.616009,-1.328507,-1.070000,1.321421
190,1.470138,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-0.402090,0.701707,1.073688,-1.606465,-1.330147,-1.086883,1.320046


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 240, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
0,-1.533996,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.060779,-1.723389,-1.790542,0.136347,0.121226,-1.445055,-0.790925
1,-1.486609,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.079132,-1.706702,-1.768299,0.149077,0.152115,-1.365321,-0.794037
2,-1.439709,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.071658,-1.690015,-1.746055,0.161806,0.183005,-1.285587,-0.797148
3,-1.393318,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.072349,-1.673328,-1.723811,0.174535,0.213895,-1.205853,-0.800259
4,-1.347462,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.073491,-1.656642,-1.701567,0.187264,0.244784,-1.126118,-0.803370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.395833,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-1.418673,1.438456,1.098176,-1.500012,-1.660268,-1.692180,1.473338
158,1.389960,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-1.470839,1.429097,1.105247,-1.521309,-1.646368,-1.657425,1.467585
159,1.384722,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-1.532363,1.419738,1.112318,-1.542605,-1.632469,-1.622671,1.461833
160,1.379646,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-1.608317,1.410379,1.119389,-1.563902,-1.618570,-1.587917,1.456081


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      256.494
1      214.974
2      273.646
3      238.045
4      254.860
        ...   
157    175.917
158    208.629
159    181.933
160    187.947
161    181.556
Name: Rio De Janeiro - Consumo de Cimento (t), Length: 162, dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Rio de-01eiro - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Rio De Janeiro - Produção de Cimento (t),Rio de Janeiro - IDH,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio de Janeiro - Desemprego
162,1.385142,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-1.590732,1.391662,1.133531,-1.606496,-1.590771,-1.518408,1.444576
163,1.390794,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-1.573361,1.382303,1.140602,-1.627793,-1.576872,-1.483654,1.438823
164,1.397927,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-1.562551,1.372945,1.147673,-1.649090,-1.562973,-1.448899,1.433071
165,1.406003,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-1.541145,1.363586,1.154744,-1.670387,-1.549074,-1.414145,1.427318
166,1.411586,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-1.535254,1.354227,1.161815,-1.691684,-1.535175,-1.379390,1.421566
167,1.415246,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-1.519728,1.344868,1.168886,-1.712981,-1.521275,-1.344636,1.415813
168,1.414247,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-1.494936,1.335510,1.175958,-1.734278,-1.507376,-1.309882,1.410061
169,1.414005,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.473917,1.320996,1.177637,-1.731580,-1.491240,-1.277229,1.403706
170,1.417239,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.434195,1.306482,1.179316,-1.728882,-1.475105,-1.244576,1.397351
171,1.414571,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.397799,1.291968,1.180995,-1.726184,-1.458969,-1.211923,1.390995


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    185.846
163    200.671
164    207.913
165    188.107
166    193.661
167    175.783
168    185.078
169    146.761
170    173.121
171    169.841
172    142.706
173    242.422
174    226.205
175    229.696
176    232.624
177    234.262
178    217.727
179    190.769
180    232.609
181    195.364
182    205.690
183    231.354
184    236.711
185    215.360
186    258.015
187    244.870
188    225.410
189    252.849
190    226.618
191    190.993
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [14]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [15]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2089684263, 2277822558, 2898425750, 538055457, 1630526047, 4233354627, 2888230847, 575712257, 3177830526, 3113755950, 2622919630, 2909073623, 3910595106, 1298604852, 3459440832, 1566510971, 3908218066, 1953599832, 3390749086, 3392239787, 4408170, 2710683579, 2004160963, 3867701419, 2452263265]


Step: 0 ___________________________________________
val_loss: 758.587890625
winner_seed: 2089684263


Step: 1 ___________________________________________
val_loss: 650.8851318359375
winner_seed: 2277822558


Step: 2 ___________________________________________
val_loss: 768.8203125


Step: 3 ___________________________________________
val_loss: 832.4859619140625


Step: 4 ___________________________________________
val_loss: 747.8405151367188


Step: 5 ___________________________________________
val_loss: 663.3236083984375


Step: 6 ___________________________________________
val_loss: 903.0841674804688


Step: 7 ___________________________________________
val_loss: 972.6277465820312


Step: 8 

In [20]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 22ms/step - loss: 91416.4844 - val_loss: 64497.7617
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 78090.2891 - val_loss: 48317.6094
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 50201.7812 - val_loss: 47390.9023
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 47453.5742 - val_loss: 40839.3438
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 41996.7891 - val_loss: 33339.1055
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 35509.4922 - val_loss: 28519.9980
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 31243.0938 - val_loss: 44888.1602
Epoch 8/10000
5/5 [==============================] - 0s 6ms/step - loss: 31696.3809 - val_loss: 33353.6367
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 31205.6406 - val_loss: 28548.3613
Epoch 10/10000
5/5 [================

5/5 [==============================] - 0s 6ms/step - loss: 1409.4347 - val_loss: 1451.2772
Epoch 79/10000
5/5 [==============================] - 0s 5ms/step - loss: 1266.4531 - val_loss: 1616.2607
Epoch 80/10000
5/5 [==============================] - 0s 5ms/step - loss: 1763.4379 - val_loss: 2736.4568
Epoch 81/10000
5/5 [==============================] - 0s 5ms/step - loss: 1437.7145 - val_loss: 1695.2435
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 1473.1610 - val_loss: 1489.8159
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 1346.0850 - val_loss: 2306.7319
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 1490.2224 - val_loss: 1794.2968
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 1373.5532 - val_loss: 2061.2380
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 2162.1877 - val_loss: 2220.8145
Epoch 87/10000
5/5 [==============================] - 0s 5ms/

Epoch 155/10000
5/5 [==============================] - 0s 5ms/step - loss: 1411.2170 - val_loss: 1888.3042
Epoch 156/10000
5/5 [==============================] - 0s 5ms/step - loss: 1917.7476 - val_loss: 1779.5872
Epoch 157/10000
5/5 [==============================] - 0s 4ms/step - loss: 1426.9624 - val_loss: 1273.1796
Epoch 158/10000
5/5 [==============================] - 0s 5ms/step - loss: 1183.0291 - val_loss: 1218.4137
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 1263.0789 - val_loss: 1247.0142
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 1028.7429 - val_loss: 1997.6240
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 1474.6305 - val_loss: 1592.5565
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 1006.0785 - val_loss: 1028.8981
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 1244.1014 - val_loss: 1392.0179
Epoch 164/10000
5/5 [================

5/5 [==============================] - 0s 5ms/step - loss: 1702.5714 - val_loss: 1904.5372
Epoch 232/10000
5/5 [==============================] - 0s 5ms/step - loss: 1581.0991 - val_loss: 2111.5320
Epoch 233/10000
5/5 [==============================] - 0s 5ms/step - loss: 1209.1536 - val_loss: 1902.5336
Epoch 234/10000
5/5 [==============================] - 0s 5ms/step - loss: 1760.6720 - val_loss: 2117.7710
Epoch 235/10000
5/5 [==============================] - 0s 5ms/step - loss: 2112.6521 - val_loss: 2068.3591
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 1440.7548 - val_loss: 2617.3240
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 1459.1549 - val_loss: 2693.7969
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 1612.7552 - val_loss: 2300.8562
Epoch 239/10000
5/5 [==============================] - 0s 4ms/step - loss: 1623.3331 - val_loss: 3437.9707
Epoch 240/10000
5/5 [==============================] 

5/5 [==============================] - 0s 4ms/step - loss: 2173.2073 - val_loss: 1774.6364
Epoch 308/10000
5/5 [==============================] - 0s 5ms/step - loss: 1607.0024 - val_loss: 1514.7787
Epoch 309/10000
5/5 [==============================] - 0s 5ms/step - loss: 1382.7909 - val_loss: 1579.8716
Epoch 310/10000
5/5 [==============================] - 0s 5ms/step - loss: 1307.3246 - val_loss: 1369.8098
Epoch 311/10000
5/5 [==============================] - 0s 6ms/step - loss: 1148.4207 - val_loss: 1560.7870
Epoch 312/10000
5/5 [==============================] - 0s 5ms/step - loss: 1294.4552 - val_loss: 1942.2169
Epoch 313/10000
5/5 [==============================] - 0s 5ms/step - loss: 1912.8757 - val_loss: 1486.0906
Epoch 314/10000
5/5 [==============================] - 0s 6ms/step - loss: 1206.0388 - val_loss: 1396.1447
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 1193.2767 - val_loss: 1536.5057
Epoch 316/10000
5/5 [==============================] 

...

5/5 [==============================] - 0s 5ms/step - loss: 1229.2850 - val_loss: 1643.9604
Epoch 767/10000
5/5 [==============================] - 0s 5ms/step - loss: 1475.4442 - val_loss: 1992.4482
Epoch 768/10000
5/5 [==============================] - 0s 5ms/step - loss: 1618.1926 - val_loss: 1201.3547
Epoch 769/10000
5/5 [==============================] - 0s 5ms/step - loss: 1274.5979 - val_loss: 1089.9996
Epoch 770/10000
5/5 [==============================] - 0s 5ms/step - loss: 1560.3649 - val_loss: 1370.7003
Epoch 771/10000
5/5 [==============================] - 0s 5ms/step - loss: 1114.4220 - val_loss: 1178.9335
Epoch 772/10000
5/5 [==============================] - 0s 5ms/step - loss: 989.0553 - val_loss: 968.2563
Epoch 773/10000
5/5 [==============================] - 0s 5ms/step - loss: 1080.9484 - val_loss: 998.9687
Epoch 774/10000
5/5 [==============================] - 0s 5ms/step - loss: 939.9402 - val_loss: 994.6443
Epoch 775/10000
5/5 [==============================] - 0s 

Epoch 843/10000
5/5 [==============================] - 0s 6ms/step - loss: 1135.3962 - val_loss: 1673.4913
Epoch 844/10000
5/5 [==============================] - 0s 5ms/step - loss: 1058.5908 - val_loss: 1413.0060
Epoch 845/10000
5/5 [==============================] - 0s 6ms/step - loss: 1039.4458 - val_loss: 1424.0615
Epoch 846/10000
5/5 [==============================] - 0s 5ms/step - loss: 1113.2120 - val_loss: 1228.7863
Epoch 847/10000
5/5 [==============================] - 0s 5ms/step - loss: 1044.1011 - val_loss: 1478.3290
Epoch 848/10000
5/5 [==============================] - 0s 5ms/step - loss: 1305.6530 - val_loss: 1662.0071
Epoch 849/10000
5/5 [==============================] - 0s 5ms/step - loss: 1078.6243 - val_loss: 1910.9426
Epoch 850/10000
5/5 [==============================] - 0s 4ms/step - loss: 1237.7585 - val_loss: 1529.2378
Epoch 851/10000
5/5 [==============================] - 0s 5ms/step - loss: 1011.7574 - val_loss: 1713.0968
Epoch 852/10000
5/5 [================

Epoch 920/10000
5/5 [==============================] - 0s 5ms/step - loss: 1090.0959 - val_loss: 1116.4929
Epoch 921/10000
5/5 [==============================] - 0s 5ms/step - loss: 994.5453 - val_loss: 1597.7157
Epoch 922/10000
5/5 [==============================] - 0s 5ms/step - loss: 912.1876 - val_loss: 1169.8291
Epoch 923/10000
5/5 [==============================] - 0s 5ms/step - loss: 937.3446 - val_loss: 1187.1949
Epoch 924/10000
5/5 [==============================] - 0s 5ms/step - loss: 1010.0114 - val_loss: 1190.9686
Epoch 925/10000
5/5 [==============================] - 0s 6ms/step - loss: 1009.1445 - val_loss: 1484.5863
Epoch 926/10000
5/5 [==============================] - 0s 5ms/step - loss: 930.6161 - val_loss: 1527.5022
Epoch 927/10000
5/5 [==============================] - 0s 5ms/step - loss: 927.1517 - val_loss: 1280.0897
Epoch 928/10000
5/5 [==============================] - 0s 6ms/step - loss: 896.1297 - val_loss: 1215.1201
Epoch 929/10000
5/5 [======================

Epoch 997/10000
5/5 [==============================] - 0s 4ms/step - loss: 1556.0320 - val_loss: 2084.4075
Epoch 998/10000
5/5 [==============================] - 0s 5ms/step - loss: 1512.8903 - val_loss: 2057.0779
Epoch 999/10000
5/5 [==============================] - 0s 5ms/step - loss: 1312.9039 - val_loss: 2223.1719
Epoch 1000/10000
5/5 [==============================] - 0s 5ms/step - loss: 1533.0670 - val_loss: 3045.9348
Epoch 1001/10000
5/5 [==============================] - 0s 5ms/step - loss: 1370.6115 - val_loss: 2273.7532
Epoch 1002/10000
5/5 [==============================] - 0s 5ms/step - loss: 1229.1074 - val_loss: 1786.0048
Epoch 1003/10000
5/5 [==============================] - 0s 5ms/step - loss: 1184.3345 - val_loss: 1872.3685
Epoch 1004/10000
5/5 [==============================] - 0s 5ms/step - loss: 1151.8075 - val_loss: 1664.3665
Epoch 1005/10000
5/5 [==============================] - 0s 5ms/step - loss: 1245.2426 - val_loss: 2072.0312
Epoch 1006/10000
5/5 [=========

5/5 [==============================] - 0s 9ms/step - loss: 1034.7994 - val_loss: 1087.3676
Epoch 1073: early stopping


In [21]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [22]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,243.566681,191.997208,191.997208,191.997208,191.997208,191.997208,243.570419,243.570435,243.570435,243.570435,243.570435,191.997208,243.570435,243.570435,243.570435,243.570435,239.841888,243.570435,243.570435,243.570435,243.570435,243.570435,243.570435,243.570435,191.997208,192.013977,204.335724,214.837555,214.83905,191.997208
Target,185.846,200.671,207.913,188.107,193.661,175.783,185.078,146.761,173.121,169.841,142.706,242.422,226.205,229.696,232.624,234.262,217.727,190.769,232.609,195.364,205.69,231.354,236.711,215.36,258.015,244.87,225.41,252.849,226.618,190.993
Error,57.720688,8.673798,15.915787,3.890213,1.663788,16.214203,58.492416,96.809433,70.449432,73.729431,100.864441,50.424789,17.365433,13.874435,10.946442,9.308441,22.114883,52.801437,10.961441,48.206436,37.880432,12.216431,6.859436,28.210434,66.017807,52.856018,21.07428,38.011444,11.778946,1.004211


In [23]:
display(mae)
display(mape)

33.877895

0.1783104

In [24]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [25]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2613.402]] - Target[2211.91]| =  Error: [[401.4922]]; MAPE:[[0.1815138]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2919.1162]] - Target[2648.3710000000005]| =  Error: [[270.74512]]; MAPE:[[0.10223081]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[1210.0208]] - Target[1398.7549999999999]| =  Error: [[188.73425]]; MAPE:[[0.13493018]]


[array([[401.4922]], dtype=float32),
 array([[270.74512]], dtype=float32),
 array([[188.73425]], dtype=float32)]

286.9905

0.13955826